# Titanic Kaggle Challenge
Notebook for Training and Evaluation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import torch as th
import numpy as np
import sys
sys.path.append('/content/drive/MyDrive/Titanic/Kaggle_Titanic')

import dataset
import evaluation
from model import Model

from tqdm import tqdm

In [2]:
use_cuda = th.cuda.is_available()
device = th.device("cuda:0" if use_cuda else "cpu")
th.backends.cudnn.benchmark = True

In [3]:
train_set = dataset.DatasetCSV("")
dataloader = th.utils.data.DataLoader(train_set,
                                      batch_size=16,
                                      shuffle=True,
                                      num_workers=2)

test_set = dataset.DatasetCSV("")

test_loader = th.utils.data.DataLoader(test_set,
                                       batch_size=16,
                                       shuffle=True)

model = Model(31)
if use_cuda:
    model = model.cuda()

In [4]:
criterion = th.nn.BCELoss()
opt = th.optim.Adam(model.parameters(), 1e-3, weight_decay=0.0001)
lr_schedule = th.optim.lr_scheduler.ExponentialLR(opt, 0.999)

Training Loop

In [ ]:
for epoch in range(100):
    iterator = tqdm(dataloader, desc=f'Epoch: {epoch}/100')
    loss_hist = []
    model.train()
    tracker = evaluation.EvaluationTracker()
    for x, y in iterator:
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        loss = criterion(y_pred, y)
        opt.zero_grad()
        loss.backward()
        opt.step()

        loss_val = loss.item()
        iterator.set_postfix({'Loss': loss_val})
        loss_hist.append(loss_val)
        tracker(y_pred=y_pred, y_true=y)
        lr_schedule.step()

    if epoch % 10 == 0:
        # Evaluation
        model.eval()
        test_tracker = evaluation.EvaluationTracker()
        with th.no_grad():
            for x, y in iterator:
                x, y = x.to(device), y.to(device)
                y_pred = model(x)
                test_tracker(y_pred=y_pred, y_true=y)

        print("Evaluation: ")
        print(test_tracker)


    print(f'Mean Loss: {np.mean(loss_hist)}')